In [7]:
import os
import subprocess
import couchdb
import pandas as pd
# from couchdb_settings import *


username = 'admin'
password = '123456'
address = 'localhost:15984'
tweets = 'view-test'

testdb = couchdb.Database('http://' + address + '/view-test')
testdb.resource.credentials = (username, password)

def set_up_db():
    os.system("chmod 0400 CCC.pem")
    arg = "ssh -i CCC.pem -L localhost:15984:172.17.0.2:5984 -N -f ec2-user@ec2-3-25-244-154.ap-southeast-2.compute.amazonaws.com"
    proc=subprocess.Popen(arg.split()).wait()
    if proc != 0:
        print("connection is already established")
        pass

class CouchDBHandler:
    def __init__(self, username, password, address, dbname):
        self.db = couchdb.Database('http://' + address + '/' + dbname)
        self.db.resource.credentials = (username, password)

set_up_db()

db_handler = CouchDBHandler(username, password, address, tweets)
print(db_handler.db)

<Database 'view-test'>


bind [127.0.0.1]:15984: Address already in use
channel_setup_fwd_listener_tcpip: cannot listen to port: 15984
Could not request local forwarding.


In [20]:
geo = {}

for item in testdb.view('geo-test/geo-count-historic', group = True):
    key = str(item.key)
    geo[key] = item.value
    
# print(geo)
    

In [8]:
languages = {}

for item in testdb.view('geo-test/lang-count', group = True, group_level = 1):
    if item.key != 'en':
        if item.key == 'in':
            languages['id'] = item.value
        else:
            languages[item.key] = item.value
            
        languages = {k:v for k, v in sorted(languages.items(), key=lambda item: item[1])[::-1][:20]}

# print(languages)

langCode = {}

with open('Data/langCode.json', 'r', encoding= 'utf-8') as f:
    for line in f:
        (val, key) = line.split()
        langCode[key] = val
        
languages = {v2: v1 for k1, v1 in languages.items() for k2, v2 in langCode.items() if k1 == k2}

print(languages)

{'Indonesian': 94, 'Spanish': 44, 'French': 31, 'Tagalog': 30, 'Japanese': 29, 'Estonian': 25, 'Portuguese': 18, 'Norwegian': 17, 'German': 17, 'Italian': 16, 'Turkish': 12, 'Arabic': 10, 'Danish': 10, 'Haitian': 10, 'Thai': 7, 'Chinese': 6, 'Dutch': 6, 'Finnish': 5, 'Slovenian': 5, 'Russian': 4}


In [9]:
def top_10_birth_country(file, N):
    data = pd.read_csv(file)
    
    match_cols = []
    new_cols = []
    col_names = data.columns
    for name in col_names:
        if name.endswith('_p'):
            match_cols.append(name)
            new_cols.append(name.strip())

    ext_data = pd.DataFrame(data[match_cols])
    ext_data.columns = new_cols
    
    grand_total = ext_data['tot_p'].sum(axis = 0)
    country_total = ext_data.sum(axis = 0)
    percentage = country_total/grand_total * 100
    
    birth_country = pd.DataFrame(country_total, columns = ['country_total'])
    birth_country['percentage'] = percentage
    
    drop_columns = ['hong_kong_sar_china_p', 'born_elsewhere_p', 'tot_p', 'os_visitors_p', 'country_birth_not_stated_p', 'australia_p', 'new_zealand_p', 'united_states_america_p', 'united_kingdom_ci_im_p', 'fiji_p', 'south_africa_p']
    
    birth_country.loc['china_excl_sars_taiwan_p'] += birth_country.loc['hong_kong_sar_china_p']
    birth_country = birth_country.T.drop(drop_columns, axis = 1)
    birth_country = birth_country.rename({'china_excl_sars_taiwan_p' : 'china_p', 'sri_lanka_p' : 'srilanka_p'}, axis = 1)
    
    country_names = []
    for item in birth_country.columns:
        item = item.split('_')
        country_names.append(item[0].capitalize())

    birth_country.columns = country_names
    birth_country = birth_country.rename({'Srilanka' : 'Sri Lanka'}, axis = 1).T
    birth_country = birth_country.sort_values(by = ['country_total'], ascending = False)[:N]
    
    return birth_country

top_10_birth_country('Data/AURIN/country_of_birth.csv', 20)

,country_total,percentage
China,174418.0,3.855451
India,160058.0,3.538028
Vietnam,78036.0,1.724960
Italy,61521.0,1.359901
Sri Lanka,52658.0,1.163987
Malaysia,45852.0,1.013543
Greece,43881.0,0.969975
Philippines,43642.0,0.964692
Pakistan,19127.0,0.422796
Germany,17511.0,0.387075


In [10]:
def top_n_lang_spoken_at_home(file_path, N):
    
    data = pd.read_csv(file_path)

    match_cols = []
    new_cols = []
    col_names = data.columns
    for name in col_names:
        if name.endswith('_P'):
            match_cols.append(name)   
            new_cols.append(name.strip())

    ext_data = pd.DataFrame(data[match_cols])
    ext_data.columns = new_cols

    SOL_tot = ext_data['SOL_Tot_P'].sum(axis = 0)
    tot = ext_data['Total_P'].sum(axis = 0)
    SOL_perc = SOL_tot/tot * 100

    drop_columns = ['SOL_Other_P', 'SOL_Samoan_P', 'SOL_Assyrian_P','SOL_Iran_Lan_Tot_P', 
                    'SOL_Irani_Lan_Othr_P', 'SOL_Se_As_A_L_Othr_P', 'SOL_Aus_Indig_Lang_P', 
                    'SOL_In_Ar_Lang_Othr_P', 'SOL_In_Ar_Lang_Tot_P', 'SOL_Se_As_A_L_Tot_P', 
                    'Language_spoken_home_ns_P', 'SOL_Tot_P', 'Total_P', 
                    'SOL_Chin_lang_Mand_P', 'SOL_Chin_lang_Other_P', 'SOL_Chin_lang_Cant_P']
    
    ext_data = ext_data.drop(drop_columns, axis = 1)
    lang_tot = ext_data.sum(axis = 0)
    
    columns = []
    for index in lang_tot.index:
        idx = index.split('_')
        if 'Se_As' in index or 'In_Ar' in index or 'Ir_Lang' in index:
            columns.append(idx[-2])
        elif 'Ir_La' in index:
            columns.append(idx[3])
        else:
            columns.append(idx[1])

    lang_tot.index = columns
    lang_data = pd.DataFrame(lang_tot, columns = ['count']).T
    lang_data = lang_data.rename({'Pe' : 'Persian'}, axis = 1).T
    lang_data['percentage_SOL'] = lang_data['count']/SOL_tot * 100
    lang_data['percentage_Total'] = lang_data['count']/tot * 100

    langdict = {k:v for v in langCode.values() for k in lang_data.index if k in v}

    idx = []
    for i in lang_data.index:
        name = langdict[i]
        idx.append(name)

    lang_data.index = idx
    lang_data = lang_data.sort_values(by = ['count'], ascending = False)[:N]

    return lang_data, SOL_perc
    
    
top_n_lang_spoken_at_home('Data/AURIN/lang_at_home.csv', 20)

(             count  percentage_SOL  percentage_Total
 Chinese     189854       16.326190          4.746370
 Greek       113424        9.753715          2.835612
 Italian     112665        9.688446          2.816637
 Vietnamese   85122        7.319930          2.128059
 Arabic       65454        5.628612          1.636357
 Hindi        31607        2.717993          0.790178
 Turkish      31257        2.687896          0.781428
 Punjabi      29517        2.538267          0.737928
 Macedonian   29378        2.526314          0.734453
 Spanish      27632        2.376170          0.690803
 Sinhala      27439        2.359573          0.685978
 Croatian     18855        1.621405          0.471377
 Maltese      17219        1.480720          0.430477
 Tamil        16907        1.453890          0.422677
 Serbian      16247        1.397135          0.406177
 German       16246        1.397049          0.406152
 Russian      15863        1.364113          0.396577
 Polish       15197        1

In [284]:
data = pd.read_csv('country_of_birth.csv')

In [285]:
match_cols = []
new_cols = []

col_names = data.columns

for name in col_names:
    if name.endswith('_p'):
        match_cols.append(name)
        new_cols.append(name.strip())

ext_data = pd.DataFrame(data[match_cols])
ext_data.columns = new_cols

In [286]:
grand_total = ext_data['tot_p'].sum(axis = 0)
country_total = ext_data.sum(axis = 0)
percentage = country_total/grand_total * 100
birth_country = pd.DataFrame(country_total, columns = ['country_total'])
birth_country['percentage'] = percentage

In [287]:
birth_country = birth_country.sort_values(by = ['country_total'], ascending = False)

In [288]:
drop_columns = ['hong_kong_sar_china_p', 'born_elsewhere_p', 'tot_p', 'os_visitors_p', 'country_birth_not_stated_p', 'australia_p', 'new_zealand_p', 'united_states_america_p', 'united_kingdom_ci_im_p', 'fiji_p', 'south_africa_p']

In [289]:
birth_country.loc['china_excl_sars_taiwan_p'] += birth_country.loc['hong_kong_sar_china_p']
birth_country = birth_country.T.drop(drop_columns, axis = 1).T[:10].T
birth_country = birth_country.rename({'china_excl_sars_taiwan_p' : 'china_p', 'sri_lanka_p' : 'srilanka_p'}, axis = 1)

In [290]:
names = []
for item in birth_country.columns:
    item = item.split('_')
    names.append(item[0].capitalize())

birth_country.columns = names

In [300]:
birth_country = birth_country.rename({'Srilanka' : 'Sri Lanka'}, axis = 1).T

In [365]:
birth_country.index

Index(['India', 'China', 'Vietnam', 'Italy', 'Sri Lanka', 'Malaysia', 'Greece',
       'Philippines', 'Pakistan', 'Germany'],
      dtype='object')

In [317]:
results = []

for i in range(len(birth_country)):
    entry = {}
    e = birth_country.iloc[i]
    entry['name'] = e.name
    entry['y'] = e.country_total
    entry['z'] = e.percentage
    results.append(entry)
print(results)

[{'name': 'India', 'y': 160058.0, 'z': 3.5380284230620616}, {'name': 'China', 'y': 174418.0, 'z': 3.85545140819977}, {'name': 'Vietnam', 'y': 78036.0, 'z': 1.7249596147775874}, {'name': 'Italy', 'y': 61521.0, 'z': 1.3599010772045204}, {'name': 'Sri Lanka', 'y': 52658.0, 'z': 1.1639874339402094}, {'name': 'Malaysia', 'y': 45852.0, 'z': 1.0135430859703463}, {'name': 'Greece', 'y': 43881.0, 'z': 0.9699747918403725}, {'name': 'Philippines', 'y': 43642.0, 'z': 0.9646917769763117}, {'name': 'Pakistan', 'y': 19127.0, 'z': 0.42279592177778097}, {'name': 'Germany', 'y': 17511.0, 'z': 0.38707478361743725}]


In [504]:
data = pd.read_csv('Data/AURIN/lang_at_home.csv')

In [505]:
match_cols = []
new_cols = []

col_names = data.columns

for name in col_names:
    if name.endswith('_P'):
        match_cols.append(name)   
        new_cols.append(name.strip())

ext_data = pd.DataFrame(data[match_cols])
ext_data.columns = new_cols

In [506]:
SOL_tot = ext_data['SOL_Tot_P'].sum(axis = 0)
tot = ext_data['Total_P'].sum(axis = 0)
SOL_perc = SOL_tot/tot * 100
SOL_perc

29.072123556525113

In [507]:
drop_columns = ['SOL_Other_P', 'SOL_Samoan_P', 'SOL_Assyrian_P','SOL_Iran_Lan_Tot_P', 'SOL_Irani_Lan_Othr_P', 'SOL_Se_As_A_L_Othr_P', 'SOL_Aus_Indig_Lang_P', 'SOL_In_Ar_Lang_Othr_P', 'SOL_In_Ar_Lang_Tot_P', 'SOL_Se_As_A_L_Tot_P', 'Language_spoken_home_ns_P', 'SOL_Tot_P', 'Total_P', 'SOL_Chin_lang_Mand_P', 'SOL_Chin_lang_Other_P', 'SOL_Chin_lang_Cant_P']

In [508]:
ext_data = ext_data.drop(drop_columns, axis = 1)

In [509]:
columns = []
lang_tot = ext_data.sum(axis = 0)
for index in lang_tot.index:
    idx = index.split('_')
    if 'Se_As' in index or 'In_Ar' in index or 'Ir_Lang' in index:
        columns.append(idx[-2])
    elif 'Ir_La' in index:
        columns.append(idx[3])
    else:
        columns.append(idx[1])

In [510]:
lang_data = pd.DataFrame(lang_tot, columns = ['count']).T
lang_data.columns = columns

In [511]:
lang_data = lang_data.rename({'Pe' : 'Persian'}, axis = 1).T
lang_data['percentage'] = lang_data['count']/SOL_tot * 100

In [519]:
langCode = {k:v for v in langCode.values() for k in lang_data.index if k in v}

In [11]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

hashtags = {}

for item in testdb.view('geo-test/hash-tags', group = True, group_level = 1):
    if item.key.lower() not in hashtags.keys():
        hashtags[item.key.lower()] = item.value
    else:
        hashtags[item.key.lower()] += item.value

hashtags = {k: v for k, v in sorted(hashtags.items(), key=lambda item: item[1])[-20:]}

hashtags

# string = ','.join(hashtags)

# wc = WordCloud(background_color="white", max_words=100)

# plt.imshow(wc.generate(string), interpolation='bilinear')
# plt.axis("off")
# plt.show()


{'it': 19,
 'victoria': 21,
 'sales': 23,
 'streetart': 24,
 'photo': 28,
 'coffee': 29,
 'spring': 30,
 'jobs': 50,
 'msfw': 53,
 'careerarc': 54,
 'hazmat': 54,
 'victraffic': 54,
 'nonstructurefire': 60,
 'incident': 65,
 'structurefire': 80,
 'hiring': 92,
 'firealarm': 95,
 'job': 103,
 'australia': 108,
 'melbourne': 564}

In [13]:
import nltk
import re
import string
from nltk.corpus import stopwords
from collections import defaultdict
from nltk.tokenize import TweetTokenizer

def topic_switch(topic):

    # if topic == 'housing':
    #     count_view = 'text/housing-count'
    #     topic_view = 'text/housing'
    # if topic == 'transportation':
    #     count_view = 'text/transportation-count'
    #     topic_view = 'text/transportation'
    # if topic == 'cost':
    #     count_view = 'text/cost-count'
    #     topic_view = 'text/cost'
        
        
    if topic == 'housing':
        count_view = 'geo-test/text-housing-count'
        topic_view = 'geo-test/text-housing'
    if topic == 'transportation':
        count_view = 'geo-test/text-transportation-count'
        topic_view = 'geo-test/text-transportation'
    if topic == 'cost':
        count_view = 'geo-test/text-cost-count'
        topic_view = 'geo-test/text-cost'
        
    return count_view, topic_view

def trend_per_topic(db, topic):

    #db = testdb
    count_view, _ = topic_switch(topic)
    
    year_topic = {}
    year_total = {}
    percent = {}

    for item in db.view(count_view, group = True, group_level = 1):
        year_topic[item.key] = item.value

    for item in db.view('time/by-year-count', group = True, group_level = 1):
        year_total[item.key] = item.value

    for key in year_topic.keys():
        if key in year_total.keys():
            percent[key] = year_topic[key]/year_total[key] * 100
            
    return year_topic, year_total, percent


def topic_wordcloud(db, topic):

    _, topic_view = topic_switch(topic)

    yearly_tweets = defaultdict(list)
    for item in db.view(topic_view):
        yearly_tweets[item.key].append(item.value)
    
    tokenizer = TweetTokenizer()
    
    for key, tweet in yearly_tweets.items():
        tweet = [' '.join(re.sub("(@[A-Za-z0-9\_]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",t).split()) for t in tweet]
        tweet = [' '.join(tweet)]
        tweet_tokens = tokenizer.tokenize(tweet[0])
        tweet_clean = []
        for word in tweet_tokens:
            if word.lower() not in stopwords.words('english') and word.lower() not in string.punctuation:
                tweet_clean.append(word.lower())
                
        yearly_tweets[key] = ' '.join(tweet_clean)
    
    return yearly_tweets

topic_wordcloud(testdb, 'housing')
     

defaultdict(list,
            {2015: 'guy refused accept private property preceded formation formal legal systems love pragmatism unsure point complex systems impossible without private property legal sys isnt initiated rule law derived respect private property fail yourdaap understanding came first law private property actually serious implications far trivial hammer tong love name love cuisine lot wasted real estate difficult replace good real estate brokers technology spring air integrity real estate integrityrealestate yarravalley yarraglen kinglake new york real estate queen dolly lenz reivstateconference 2015 dollylenzrealestate wbpproperty'})

In [55]:
from textblob import TextBlob

def topic_sentiment(db, topic):

    yearly_tweets = topic_wordcloud(db, topic)
    
    yearly_sentiment = {}
    for key, value in yearly_tweets.items():
        blob = TextBlob(value)
        for sentence in blob.sentences:
            sentiment = sentence.sentiment.polarity
            yearly_sentiment[key] = sentiment
    return yearly_sentiment

topic_sentiment(testdb, 'transportation')

{2015: -0.020908143939393962}